# <div align="center" style="color: #ff5733;">TSA Closure Second Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID c7c5ffa1-17e5-4287-a70b-69d0aa795cbe successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-05-16,550201


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\March24&April24 Ongoing Batch\SecondNotification_Files\Type 1 & 2 - April.xlsx")
d1.shape

(14371, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	14371


,Customer_Id,ACTION_TYPE
0,2352790,TYPE 1
1,2363211,TYPE 1
2,2317709,TYPE 1
3,2367252,TYPE 1
4,2349365,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\March24&April24 Ongoing Batch\SecondNotification_Files\Type 3 - April.xlsx")
d2.shape

(2511, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	2511


,Customer_Id,ACTION_TYPE
0,2197600,TYPE 3
1,2215689,TYPE 3
2,2279322,TYPE 3
3,2276454,TYPE 3
4,2247591,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(16882, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    14326
TYPE 3     2511
TYPE 2       45
Name: count, dtype: int64

In [15]:
result[result['Customer_Id'] == '2197600']

,Customer_Id,ACTION_TYPE
14371,2197600,TYPE 3


In [12]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16882 entries, 0 to 16881
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  16882 non-null  int64 
 1   ACTION_TYPE  16882 non-null  object
dtypes: int64(1), object(1)
memory usage: 263.9+ KB


In [13]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16882 entries, 0 to 16881
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  16882 non-null  object
 1   ACTION_TYPE  16882 non-null  object
dtypes: object(2)
memory usage: 263.9+ KB


In [14]:
result.to_csv("temp.csv", index = False)

In [16]:
a = tuple(result['Customer_Id'])
a[:5]

('2352790', '2363211', '2317709', '2367252', '2349365')

In [20]:
d = '2024-04-23'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-02-23'
# Calculate D-180 days
d_minus_180_days = '2023-10-26'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-05-16 00:00:00
D-60 day from the current date - 1: 2024-02-23
D-180 day from the current date -1: 2023-10-26
D-1 day from the current date: 2024-05-16
D-60 day from the current date: 2024-02-23
D-180 day from the current date: 2023-10-26


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [18]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe()
dfd.shape

(8886, 29)

In [21]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [22]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-05-16'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('2352790', '2363211', '2317709', '2367252', '2349365', '2313900', '2360327', '2373619', '2368104', '2403316', '2381240', '2357030', '1432588', '2391318', '2135002', '2351837', '2359223', '2378978', '2404414', '2355589', '2362364', '2398075', '2384226', '2375514', '2351082', '2372641', '2387692', '2390319', '2365873', '2389416', '2378747', '2366609', '2329364', '2369200', '2374355', '2362924', '2373320', '2358066',

In [23]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 83d0c642-d4a6-43c6-836d-8839e94f265d successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [24]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
988,2382387,2024-01-28,60823823870006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,1,0,0,2024-04-17,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,0,NaT,NaT,TYPE 1
8876,2380171,2024-01-27,60823801710004,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-27,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,1,NaT,2024-04-26,TYPE 1
3414,2381390,2024-01-27,60823813900007,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-27,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,0,NaT,NaT,TYPE 1
4940,1286572,2024-02-20,60812865720008,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,2022-11-11,N,0,0.0,0,0,1,0,0,2024-02-23,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,0,NaT,NaT,TYPE 1
7112,2390481,2024-02-05,60823904810001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-03-30,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,1,NaT,2024-05-06,TYPE 1
490,2403831,2024-02-18,60824038310005,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-02-18,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,0,NaT,NaT,TYPE 1
8714,2399416,2024-02-13,60823994160007,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-20,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,1,NaT,2024-05-14,TYPE 1
7254,2390306,2024-02-09,60823903060006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-05-16,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,1,NaT,2024-05-05,TYPE 1
7601,2120982,2023-07-05,60821209820009,0.0,Active,0.09,0.0,0.0,0.0,0,2023-09-30,2023-09-14,N,0,0.0,0,0,0,0,0,2023-09-14,2024-05-16,NaT,NaN,2024-05-16,Batch_1,May,0,0,NaT,NaT,TYPE 3
4750,2393569,2024-02-08,60823935690009,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-28,NaT,NaT,NaN,2024-05-16,Batch_1,May,0,1,NaT,2024-05-09,TYPE 1


In [25]:
df.shape

(8886, 32)

In [26]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       7522
TYPE 3       1062
NO ACTION     282
TYPE 2         20
Name: count, dtype: int64

In [27]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\March24&April24 Ongoing Batch\20240517_TSA_Account_Closure_2ndtNotification.xlsx", sheet_name='TSA_Closure_2ndNotification', index = False)